In [1]:
images_dimensions = 128
attribute = 'initial-set--Type-4-17'
totalInputImages = 5000
BATCH_SIZE = 100
latent_dim = 100

In [2]:
IMAGES_FOLDER = './images/'
folderDimensionsSufix = str( images_dimensions ) + 'x' + str( images_dimensions )

padronizedFolder = f'{IMAGES_FOLDER}padronized--{attribute}--{folderDimensionsSufix}--{totalInputImages}--bs-{BATCH_SIZE}--ls-{latent_dim}/'

In [3]:
trainedModelFolder = f'./trained-model--{attribute}--{folderDimensionsSufix}--{totalInputImages}--bs-{BATCH_SIZE}--ls-{latent_dim}/'
generatedFolder = trainedModelFolder + 'generated/'

In [4]:
# example of calculating the frechet inception distance in Keras
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize

# Daniel
import os
from tqdm import tqdm
import cv2
import numpy as np
import time
import gc

lastTimeNotebook = time.time()

def scale_images(images, new_shape):
    
    images_list = list()
    for image in images:
        
        new_image = resize(image, new_shape, 0)
        images_list.append(new_image)
        
    return asarray(images_list)

def calculate_fid(model, images1, mu2, sigma2):
    
    # calculate activations
    act1 = model.predict(images1)
    
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    
    # Daniel
    sigma1 = np.array(sigma1, ndmin=2) # https://stackoverflow.com/questions/31698242/python-how-can-i-force-1-element-numpy-arrays-to-be-two-dimensional
    
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    
    return fid

def loadImages(folder):
    
    path, dirs, files = next( os.walk( folder ) )

    images = []
    contador = 0
    for file in tqdm(files):

        image = cv2.imread( folder + file )
        images.append( image )
        contador += 1
        
    images = np.asarray(images)
    return images

def compareTwoGroups(model, images1, mu2, sigma2):

    # convert integer to floating point values
    images1 = images1.astype('float32')
    
    # resize images
    images1 = scale_images(images1, (299,299,3))

    # pre-process images
    images1 = preprocess_input(images1)

    # fid between images1 and images2
    fid = calculate_fid(model, images1, mu2, sigma2)
    
    return fid

# ---

print( '\nProcessing...' )

# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

# Real Images used in training
realImages = loadImages(padronizedFolder)
gc.collect()
realImages = realImages.astype('float32')
gc.collect()
realImages = scale_images(realImages, (299,299,3))
gc.collect()
realImages = preprocess_input(realImages)
gc.collect()

act2 = model.predict(realImages)
gc.collect()
mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
gc.collect()
sigma2 = np.array(sigma2, ndmin=2) # https://stackoverflow.com/questions/31698242/python-how-can-i-force-1-element-numpy-arrays-to-be-two-dimensional
gc.collect()

epochsFolders = os.listdir(generatedFolder)
totalEpochs = len(epochsFolders)

bestEpoch = ''
bestFID = 9999

lastTime = time.time()
for i in range(1, totalEpochs + 1):

    print('\n--------------------------------\n')
    print(f'Epoch: {i}')
    
    epoch = 'epoch-' + str(i)
    generatedImages = loadImages( f'{generatedFolder}/{epoch}/' )

    fid = compareTwoGroups(model, generatedImages, mu2, sigma2)

    print()
    print('FID: %.4f' % fid)

    if fid < bestFID:
        bestFID = fid
        bestEpoch = epoch
        
    # Time
    now = time.time()
    interval = now - lastTime
    lastTime = now
    print('Time: %.2f min' % (interval / 60))
    print()
    print('-- Partial --')
    print(f'Best Epoch: {bestEpoch}')
    print('Best FID: %.4f' % bestFID)


print( '\n----------------------------------------------------' )
print( 'Finalized' )

# Time
now = time.time()
interval = now - lastTimeNotebook
print(f'Notebook Time: {(interval / 60):.2} min')

print(f'Best Epoch: {bestEpoch}')
print('Best FID: %.4f' % bestFID)

Using TensorFlow backend.



Processing...


100%|██████████| 30/30 [00:00<00:00, 163.62it/s]


--------------------------------

Epoch: 1



 50%|█████     | 15/30 [00:00<00:00, 148.31it/s]

...

 63%|██████▎   | 19/30 [00:00<00:00, 189.36it/s]


FID: 82.5364
Time: 0.13 min

-- Partial --
Best Epoch: epoch-194
Best FID: 72.3574

--------------------------------

Epoch: 199


100%|██████████| 30/30 [00:00<00:00, 188.39it/s]


FID: 79.5262
Time: 0.13 min

-- Partial --
Best Epoch: epoch-194
Best FID: 72.3574

--------------------------------

Epoch: 200



FID: 77.1812
Time: 0.13 min

-- Partial --
Best Epoch: epoch-194
Best FID: 72.3574

----------------------------------------------------
Finalized
Notebook Time: 2.8e+01 min
Best Epoch: epoch-194
Best FID: 72.3574
